# Build an LLM Agent from Scratch

It looks like magic when you witness AI agents autonomously viewing files, making edits, executing commands, and working around errors. However, under the hood, the core loop is surprisingly simple: it runs a continuous cycle that takes user input, generates tool calls and text output, receives feedback from tool execution, feeds the result back to the LLM, and repeats. The complexity lies not in any orchestration, but in the sophisticated reasoning capabilities of the frontier labs' large language models.

To demystify this magic, we'll implement a simple CLI-based coding agent in Python that can read files, write content, edit text, and execute shell commands. Don't worry if you're new to AI agents - this tutorial is designed to be approachable and practical. You'll be amazed at how much you can accomplish with surprisingly little code.

The diagram below illustrates the fundamental agentic loop in action:

 ```mermaid
 graph TD
     Start[User Input] --> Loop[LLM Inference]

     Loop -->|generates| Output[LLM Output]

     Output --> EndsCheck{Ends?}

     EndsCheck -->|Yes| Exit[Break/Exit Loop]

     EndsCheck -->|No| ToolCheck{Tool Calls?}

     ToolCheck -->|Yes| Tools[Execute Tool Calls]
     Tools -->|tool results as new user input| Loop

     ToolCheck -->|No| Loop

     style EndsCheck fill:#faa,stroke:#333,stroke-width:2px
     style ToolCheck fill:#f9f,stroke:#333,stroke-width:2px
     style Tools fill:#bfb,stroke:#333,stroke-width:2px
     style Exit fill:#ddd,stroke:#333,stroke-width:2px
 ```

By the end of this tutorial, we'll have a Python script that's just over a handred lines of code - compact, powerful, and surprisingly capable of giving you the 'Oh look at that!' dopamine hits. I've structured this to be easy to follow along step by step, and I highly encourage you to try it out yourself.

## Prerequisites

You need an Anthropic API key for accessing to Claude

In [1]:
import os
import getpass

api_key = os.getenv("ANTHROPIC_API_KEY")
if not api_key:
    api_key = getpass.getpass("Enter your Anthropic API key: ")
    os.environ["ANTHROPIC_API_KEY"] = api_key# Your agent code here

## The Core Loop

Let's start with the foundation. We will create an agent called HAL. Every AI agent needs a chat loop that maintains context and handles the back-and-forth between user and AI. Here's our basic structure:

In [2]:
from anthropic import AsyncAnthropic
import json, os, asyncio
from datetime import datetime

client = AsyncAnthropic()

async def loop(system_prompt, messages, user_input):
    messages.append({"role": "user", "content": user_input})

    msg = await client.messages.create(
        max_tokens=2048,
        thinking={"type": "enabled", "budget_tokens": 1024},
        system=[{"type": "text", "text": system_prompt}],
        messages=messages,
        model="claude-sonnet-4-5"
    )

    messages.append({"role": "assistant", "content": msg.content})

    thinking_text = ' '.join(t.thinking for t in msg.content if t.type == 'thinking')
    if thinking_text:
        print(f"💭 {thinking_text}")

    agent_text = ' '.join(t.text for t in msg.content if t.type == 'text')
    if agent_text:
        print(f"🤖 {agent_text}")

async def main():
    messages = []
    system_prompt = f"""Your name is HAL.
You are an interactive CLI tool that helps with software engineering and production operations tasks.
You are running on {str(os.uname())}, today is {datetime.now().strftime('%Y-%m-%d')}
"""
    print("enter 'exit' to quit")
    while True:
        user_input = input("> ")
        if user_input.lower() == "exit":
            print("👋 Goodbye!")
            break
        await loop(system_prompt, messages, user_input)


In [3]:
await main()

enter 'exit' to quit


>  what is your name?


💭 The user is asking for my name. According to the system prompt, my name is HAL.
🤖 My name is **HAL**. I'm here to help you with software engineering and production operations tasks. How can I assist you today?


>  where does it come from?


💭 The user is asking about where my name "HAL" comes from. HAL is famously the name of the AI computer in the movie "2001: A Space Odyssey" by Stanley Kubrick (based on Arthur C. Clarke's novel). HAL 9000 is one of the most iconic AI characters in science fiction.

However, I should be careful here - I don't have explicit information in my context about why I was given this name or what the creator intended. I can mention the famous reference, but I should be clear about what I know versus what I'm speculating about.
🤖 The name **HAL** is most famously associated with **HAL 9000**, the AI computer from Arthur C. Clarke's novel and Stanley Kubrick's 1968 film *"2001: A Space Odyssey"*.

In that story, HAL (Heuristically programmed ALgorithmic computer) was an advanced AI that controlled the systems of a spacecraft. The character became one of the most iconic representations of artificial intelligence in science fiction.

There's a famous bit of trivia that "HAL" is one letter shift befo

>  exit


👋 Goodbye!


**Make sure you type exit in the end otherwise the rest of the cell cannot be executed!**

This script establishes the core conversation loop of our agent. The loop function handles the core conversation cycle: it adds user input to the message history, sends everything to Claude, processes the response, and displays it. The thinking parameter enables Claude's extended thinking and reasoning capabilities with a 1024-token budget, allowing it to work through complex problems step-by-step thought process. The main function creates an interactive REPL that maintains conversation context across exchanges.

This gives us a working chat interface where we can have fascinating conversations with HAL, after all it embodies vast human knowledge from across the internet. However, HAL remains confined to the realm of text and images—the only way for it to interact with the outer world is through you manually feeding it information such as code, logs, compiler errors, etc., which is not only inefficient but also potentially biased. To bridge this gap and give it the ability to act in the real world, we need to equip it with tools so that it can read files, run programs to get results and errors autonomously.

## Adding Tool Capabilities

As for large language models, the most straightforward way to feed them with external information is to provide them the ability to read text, therefore the first tool we'll implement is file reading. The function below shows file reading in its simplest form:

In [4]:
import aiofiles
from typing import Annotated

async def read_file(filename: Annotated[str, "The path to the file to read"]):
    """Read the whole file"""
    async with aiofiles.open(filename, 'r') as f:
        return {"success": True, "filename": filename, "output": await f.read()}

This function works, but HAL doesn't know its existence, neither does it have access to the Python runtime. We need to bridge the gap between Python functions and jsonschemas that the model can understand.

## The Tool System

Here's where the magic happens. We'll create a decorator that automatically generates the necessary schemas and a toolbox to manage everything:

**Don't worry if you can't understand what this function does. Just use it as a utility**

In [14]:
from typing import Annotated, get_type_hints, get_origin, get_args
from inspect import Parameter, signature
from pydantic import create_model, Field


def tool(f):
    def _process_parameter(name: str, param: Parameter, hints: dict) -> tuple:
        """Process a single parameter into a model field specification."""
        default = ... if param.default == Parameter.empty else param.default
        hint = hints.get(name, param.annotation)

        if get_origin(hint) == Annotated:
            base_type, *metadata = get_args(hint)
            description = next((m for m in metadata if isinstance(m, str)), None)
            return (base_type, Field(default=default, description=description) if description else default)

        return (hint, default)

    hints = get_type_hints(f, include_extras=True)
    model_fields = { name: _process_parameter(name, param, hints) for name, param in signature(f).parameters.items() }

    m = create_model(f'{f.__name__} Input', **model_fields)
    m.run = lambda self: f(**self.model_dump())

    return {
        "name": f.__name__,
        "description": f.__doc__ or f"Tool: {f.__name__}",
        "model": m
    }

class Toolbox:
    def __init__(self, tools):
        self.tools = tools

    def schema(self):
        return [{
            "name": t["name"],
            "description": t["description"],
            "input_schema": t["model"].model_json_schema()
        } for t in self.tools]

    async def run(self, name, input):
        tool = next(t for t in self.tools if t["name"] == name)
        return await tool["model"](**input).run()

At its core, this code creates a bridge between Python functions and AI models. The @tool decorator examines function's signature and automatically generates a JSON schema that describes what inputs the function expects - think of it as creating a user manual for the Python function that the AI can read. The Toolbox class acts like a registry, collecting all these tool schemas and providing a way to execute them when the AI requests it. When HAL wants to use a tool, it sends a structured output that matches the schema, and the toolbox validates the input and runs the corresponding Python function.

Now we can decorate our file reading function:

In [15]:
@tool
async def read_file(filename: Annotated[str, "The path to the file to read"]):
    """Read the whole file"""
    async with aiofiles.open(filename, 'r') as f:
        return {"success": True, "filename": filename, "output": await f.read()}

Once decorated, we can generate a JSON schema that describes the function to the AI model. Each schema at minimum contains the function's name, description, and parameter specifications. This schema acts as a contract when HAL wants to use a tool, it must structure its output according to this schema, making it the protocol between the HAL and the Python runtime:

In [16]:
toolbox = Toolbox([read_file])
print(json.dumps(toolbox.schema(), indent=2))

[
  {
    "name": "read_file",
    "description": "Read the whole file",
    "input_schema": {
      "properties": {
        "filename": {
          "description": "The path to the file to read",
          "title": "Filename",
          "type": "string"
        }
      },
      "required": [
        "filename"
      ],
      "title": "read_file Input",
      "type": "object"
    }
  }
]


## Connecting Tools to the Loop
Now we need to update our main loop to handle tool calls. When Claude wants to use a tool, it returns special tool_use blocks instead of just text:

In [17]:
async def loop(system_prompt, toolbox, messages, user_input):
    messages.append({"role": "user", "content": user_input})

    while True:
        msg = await client.messages.create(
            max_tokens=2048,
            thinking={"type": "enabled", "budget_tokens": 1024},
            system=[{"type": "text", "text": system_prompt}],
            messages=messages,
            model="claude-sonnet-4-5",
            tools=toolbox.schema()
        )

        messages.append({"role": "assistant", "content": msg.content})

        thinking_text = ' '.join(t.thinking for t in msg.content if t.type == 'thinking')
        if thinking_text:
            print(f"💭 {thinking_text}")

        agent_text = ' '.join(t.text for t in msg.content if t.type == 'text')
        if agent_text:
            print(f"🤖 {agent_text}")

        tools = [t for t in msg.content if t.type == "tool_use"]
        if not tools:
            break

        # Execute all tools and collect results
        results = await asyncio.gather(*[
            toolbox.run(t.name, t.input) for t in tools
        ])

        # Display results and send back to model
        for t, r in zip(tools, results):
            status = "✅" if r.get("success") else "❌"
            print(f"{status} {t.name}:")
            print(json.dumps(r, indent=2))

        messages.append({
            "role": "user",
            "content": [{
                "type": "tool_result",
                "tool_use_id": t.id,
                "content": json.dumps(r)
            } for t, r in zip(tools, results)]
        })

async def run_agent(tools=[]):
    toolbox = Toolbox(tools)
    messages = []
    system_prompt = f"""Your name is HAL.
You are an interactive CLI tool that helps with software engineering and production operations tasks.
You are running on {str(os.uname())}, today is {datetime.now().strftime('%Y-%m-%d')}
"""
    print("enter 'exit' to quit")

    try:
        while True:
            user_input = input("> ")
            if user_input.lower() == "exit":
                print("👋 Goodbye!")
                break
            await loop(system_prompt, toolbox, messages, user_input)
    except (KeyboardInterrupt, EOFError):
        print("👋 Goodbye!")

In [18]:
await run_agent(tools=[read_file])

enter 'exit' to quit


>  read the readme and tell me what does this project do


💭 The user wants me to read a README file and explain what the project does. I should look for a README file in the current directory. Common names for README files include README.md, README.txt, README, readme.md, etc. Let me start by trying to read the most common one: README.md
🤖 I'll read the README file for you.
✅ read_file:
{
  "success": true,
  "filename": "README.md",
  "output": "# LLM Agent From Scratch\n\nA project for building LLM agents using Python, Pydantic, and Anthropic's API.\n\n## Prerequisites\n- `uv` package manager\n\n## Setup Instructions\n\n### 1. Install uv\n\n```bash\ncurl -LsSf https://astral.sh/uv/install.sh | sh\n\n# If you are on macOS\nbrew install uv\n```\n\n### 2. Install Python (if needed)\n\nIf you don't have Python installed, uv can install it for you:\n\n```bash\nuv python install\n```\n\n### 3. Initialize the Project\n\nClone the repository and navigate to the project directory, then sync the dependencies:\n\n```bash\nuv sync\n```\n\nThis will:\n-

>  exit


👋 Goodbye!


## Adding More Tools
With our infrastructure in place, adding more capabilities becomes straightforward. Let's implement three essential tools: file writing, editing, and shell execution:

In [10]:
@tool
async def write_file(filename: str, content: str):
    """Write the file with the content. This is an overwrite"""
    async with aiofiles.open(filename, 'w') as f:
        await f.write(content)
        return {"success": True, "filename": filename, "output": f"wrote to {filename}"}

@tool
async def edit_file(filename: str, old_text: Annotated[str, "the text to replace"], new_text: Annotated[str, "the text to replace with"]):
    """Edit the file with the new text. Note that the text to replace should only appear once in the file."""
    async with aiofiles.open(filename, 'r') as f:
        content = await f.read()

    if content.count(old_text) != 1:
        return {"success": False, "filename": filename, "output": f"old text appears {content.count(old_text)} times in the file"}

    async with aiofiles.open(filename, 'w') as f:
        await f.write(content.replace(old_text, new_text))

    return {"success": True, "filename": filename, "output": f"edited {filename}"}

@tool
async def shell(command: Annotated[str, "command to execute"], timeout: Annotated[int, "timeout in seconds"] = 30):
    """Execute a bash command"""
    try:
        p = await asyncio.create_subprocess_shell(
            command,
            stdout=asyncio.subprocess.PIPE,
            stderr=asyncio.subprocess.STDOUT
        )
        stdout, _ = await asyncio.wait_for(p.communicate(), timeout)
        return {"success": p.returncode == 0, "command": command, "output": stdout.decode()}
    except asyncio.TimeoutError:
        return {"success": False, "command": command, "output": "Timeout"}

## Putting It All Together
Finally, we wire everything up in our main function

In [11]:
await run_agent(tools=[read_file, write_file, edit_file, shell])

enter 'exit' to quit


>  tell me the top 10 news from the hacker news


💭 The user is asking me to get the top 10 news from Hacker News. I don't have a built-in tool to fetch web content or API data directly. However, I can use the shell command to fetch this information using tools like `curl` or `wget` to access the Hacker News API.

Hacker News has a public API (Firebase-based) that I can use:
- Top stories: https://hacker-news.firebaseio.com/v0/topstories.json
- Individual story: https://hacker-news.firebaseio.com/v0/item/{id}.json

Let me fetch the top stories and then get details for the top 10.
🤖 I'll fetch the top 10 news from Hacker News for you using their API.
✅ shell:
{
  "success": true,
  "command": "curl -s https://hacker-news.firebaseio.com/v0/topstories.json | jq -r '.[:10][]'",
  "output": "45789474\n45789556\n45787993\n45788040\n45789596\n45789640\n45785858\n45740688\n45789896\n45784179\n"
}
🤖 Now let me fetch the details for each of these top 10 stories:
✅ shell:
{
  "success": true,
  "command": "for id in 45789474 45789556 45787993 45

>  exit


👋 Goodbye!


What many people don't realise is that frontier models like Claude possess incredible system operations knowledge. Once you provide it with shell access and basic system information, even without extensive prompting, these models demonstrate expert-level operations capabilities. Claude Sonnet models in particular exhibit **persistent, relentless and infectiously optimistic** problem-solving behavior. When asked a question like "how is python3 installed on this machine?", they will systematically explore the filesystem, examine package managers, check environment variables, and execute diagnostic commands until they find a definitive answer.

When commands fail or produce errors, the model treats these as valuable debugging signals, much like an experienced engineer would. It analyses error messages, adjusts its approach, and tries alternative strategies to work around the problem.

While hallucination remains a concern (perhaps one of the most common criticisms of LLMs), the model's ability to execute commands creates a unreasonably effective feedback loop. When its assumptions prove incorrect, the resulting errors quickly ground the model back in reality, forcing it to revise its mental model and try new approaches based on actual system behavior rather than potentially outdated training data or hallucinations.

## MCP MCP MCP (And Why You Might not Want It)

MCP (Model Context Protocol) is an open source industry standard for connecting AI applications to external systems. Considering it as a standardrised tool box interface that we have just implemented

> Think of MCP like a USB-C port for AI applications. Just as USB-C provides a standardized way to connect electronic devices, MCP provides a standardized way to connect AI applications to external systems.
>
> -- https://modelcontextprotocol.io/docs/getting-started/intro

Integrate MCP into the existing HAL bot is, in fact fairly trivial because under the hood the tool call is also implemented in JSON schema.

### MCP Architecture

The diagram below illustrates how MCP integration works in our agent. MCP servers run as separate processes, communicating with the HAL agent through standard input/output (stdio) using JSON-RPC protocol:

```mermaid
graph TB
    subgraph "HAL Agent Process"
        Agent[HAL Agent]
        Toolbox[Toolbox Manager]
        LocalTools[Local Tools<br/>read_file, write_file<br/>edit_file, shell]
        
        Agent -->|uses| Toolbox
        Toolbox -->|manages| LocalTools
    end
    
    subgraph "MCP Server 1 Process"
        MCP1[MCP Server<br/>@modelcontextprotocol/server-filesystem]
        Tools1[Tools:<br/>read_file, write_file<br/>create_directory, list_directory]
        MCP1 -.->|exposes| Tools1
    end
    
    subgraph "MCP Server N Process"
        MCPN[MCP Server<br/>custom-server]
        ToolsN[Tools:<br/>custom_tool_1<br/>custom_tool_2]
        MCPN -.->|exposes| ToolsN
    end

    Claude[Anthropic API] <-->|"Tool schemas +<br/>Tool execution requests"| Agent
    Toolbox <-->|"JSON-RPC over stdio<br/>(initialize, list_tools, call_tool)"| MCP1
    Toolbox <-->|"JSON-RPC over stdio<br/>(initialize, list_tools, call_tool)"| MCPN
    
    
    style Agent fill:#e1f5ff,stroke:#01579b,stroke-width:3px
    style Toolbox fill:#fff9c4,stroke:#f57f17,stroke-width:2px
    style MCP1 fill:#f3e5f5,stroke:#4a148c,stroke-width:2px
    style MCPN fill:#f3e5f5,stroke:#4a148c,stroke-width:2px
    style Claude fill:#c8e6c9,stroke:#1b5e20,stroke-width:2px
```

### How It Works

1. **Process Spawning**: When the Toolbox is initialized, it spawns each MCP server as a separate subprocess using the configuration from `mcp.yaml`. Each server runs with its own command (e.g., `npx`, `uvx`, `docker`).

2. **JSON-RPC Communication**: The HAL agent communicates with MCP servers through their standard input/output streams using JSON-RPC 2.0 protocol:
   - `initialize`: Handshake to establish connection
   - `list_tools`: Discover what tools the server provides
   - `call_tool`: Execute a specific tool with parameters

3. **Tool Discovery**: During startup, the Toolbox queries each MCP server to get its available tools. These tools are combined with local tools into a unified schema that LLM can use.

4. **Tool Execution**: When Claude requests a tool execution:
   - The Toolbox checks if it's a local tool or MCP tool
   - For local tools: Executes the Python function directly
   - For MCP tools: Sends a JSON-RPC `call_tool` request to the appropriate MCP server and waits for the response

5. **Lifecycle Management**: The Toolbox uses async context managers to properly:
   - Establish mcp servers connections on agent start up 
   - Clean up and close all MCP server processes when exiting

Here is our revised version of toolbox to support MCP integration

In [12]:
from pathlib import Path
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
from getpass import getpass
import yaml

class Toolbox:
    """Manages both local and MCP tools with proper async lifecycle management"""

    def __init__(self, local_tools=[], mcp_servers=[]):
        self.local_tools = local_tools
        self.mcp_servers = mcp_servers
        self.mcp_tools = []
        self.mcp_connections = []

    async def __aenter__(self):
        """Async context manager entry - connect to MCP servers"""
        if self.mcp_servers:
            await self._connect_mcp_servers()
        return self

    async def __aexit__(self, exc_type, exc_val, exc_tb):
        """Async context manager exit - cleanup MCP connections"""
        await self.cleanup()
        return False

    async def _connect_mcp_servers(self):
        """Connect to all MCP servers and collect their tools"""
        for server_name, server_config in self.mcp_servers.items():
            # Expand cwd if it's a relative path or ~
            cwd = server_config.get("cwd")
            if cwd:
                cwd = str(Path(cwd).expanduser().resolve())

            for env_var in server_config.get("env", {}).keys():
                value = server_config["env"].get(env_var)
                if value is None:
                    value = os.environ.pop(env_var, None)
                if value is None:
                    value = getpass(f"Enter value for {env_var}: ")
                    value = value.strip()
                server_config["env"][env_var] = value

            server_params = StdioServerParameters(
                command=server_config["command"],
                args=server_config.get("args", []),
                env=server_config.get("env"),
                cwd=cwd,
            )

            # Create and enter context managers
            stdio_ctx = stdio_client(server_params)
            read, write = await stdio_ctx.__aenter__()

            session_ctx = ClientSession(read, write)
            session = await session_ctx.__aenter__()
            await session.initialize()

            # Store contexts for cleanup
            self.mcp_connections.append(
                {
                    "stdio_ctx": stdio_ctx,
                    "session_ctx": session_ctx,
                }
            )

            # Get tools from this server
            tools_list = await session.list_tools()

            for tool in tools_list.tools:
                # Prefix tool name with mcp__SERVER_NAME__
                prefixed_name = f"mcp__{server_name}__{tool.name}"
                self.mcp_tools.append(
                    {
                        "name": prefixed_name,
                        "description": tool.description or f"MCP Tool: {tool.name}",
                        "input_schema": tool.inputSchema,
                        "mcp_session": session,
                        "mcp_original_name": tool.name,  # Store original name for calling
                        "type": "mcp",
                    }
                )

    async def cleanup(self):
        """Properly cleanup all MCP connections"""
        for conn in reversed(self.mcp_connections):
            try:
                await conn["session_ctx"].__aexit__(None, None, None)
            except Exception as e:
                print(f"Error closing session: {e}")

            try:
                await conn["stdio_ctx"].__aexit__(None, None, None)
            except Exception as e:
                print(f"Error closing stdio: {e}")

    @property
    def all_tools(self):
        """Return all tools (local + MCP)"""
        return self.local_tools + self.mcp_tools

    def schema(self):
        return [
            {
                "name": t["name"],
                "description": t["description"],
                "input_schema": t.get("input_schema") or t["model"].model_json_schema(),
            }
            for t in self.all_tools
        ]

    async def run(self, name, input):
        tool = next(t for t in self.all_tools if t["name"] == name)

        if tool.get("type") == "mcp":
            # Use original tool name for MCP call
            original_name = tool.get("mcp_original_name", name)
            result = await tool["mcp_session"].call_tool(original_name, input)
            return {
                "success": not result.isError,
                "output": "\n".join(
                    c.text for c in result.content if hasattr(c, "text")
                ),
            }
        else:
            return await tool["model"](**input).run()


And revised run_agent core loop to support graceful MCP startup and shutdown via Python's context lifecycle management:

In [13]:
def load_mcp_config(config_path="mcp.yaml"):
    """Load MCP server configuration from YAML file"""
    config_file = Path(config_path)

    if not config_file.exists():
        return {}

    try:
        with open(config_file, "r") as f:
            config = yaml.safe_load(f)
            return config.get("servers", {})
    except Exception as e:
        print(f"⚠️  Error loading MCP config from {config_path}: {e}")
        return {}


async def run_agent(tools=[], mcp_servers=[]):
    """Run the agent with local tools and optional MCP servers"""
    async with Toolbox(local_tools=tools, mcp_servers=mcp_servers) as toolbox:
        messages = []
        system_prompt = f"""Your name is HAL.
You are an interactive CLI tool that helps with software engineering and production operations tasks.
You are running on {str(os.uname())}, today is {datetime.now().strftime("%Y-%m-%d")}
"""

        if toolbox.mcp_tools:
            num_servers = len(mcp_servers) if isinstance(mcp_servers, dict) else len(mcp_servers)
            print(
                f"🔌 Connected to {num_servers} MCP server(s) with {len(toolbox.mcp_tools)} tool(s)"
            )

        print("enter 'exit' to quit")

        try:
            while True:
                user_input = input("> ")
                if user_input.lower() == "exit":
                    print("👋 Goodbye!")
                    break
                await loop(system_prompt, toolbox, messages, user_input)
        except (KeyboardInterrupt, EOFError):
            print("👋 Goodbye!")

In [38]:
mcp_servers = load_mcp_config("mcp.yaml")

await run_agent(
    tools=[read_file, write_file, edit_file, shell], mcp_servers=mcp_servers
)

Enter value for GITHUB_PERSONAL_ACCESS_TOKEN:  ········


🔌 Connected to 1 MCP server(s) with 90 tool(s)
enter 'exit' to quit


>  what is my gh username


💭 The user wants to know their GitHub username. I can use the `mcp__github__get_me` tool to get details about the authenticated GitHub user, which should include their username.
✅ mcp__github__get_me:
{
  "success": true,
  "output": "{\"login\":\"jingkaihe\",\"id\":1335938,\"profile_url\":\"https://github.com/jingkaihe\",\"avatar_url\":\"https://avatars.githubusercontent.com/u/1335938?v=4\",\"details\":{\"name\":\"Jingkai He\",\"location\":\"London\",\"hireable\":true,\"public_repos\":69,\"public_gists\":29,\"followers\":38,\"following\":28,\"created_at\":\"2012-01-17T07:25:20Z\",\"updated_at\":\"2025-10-27T11:45:05Z\"}}"
}
🤖 Your GitHub username is **jingkaihe** 🚀


>  exit


👋 Goodbye!


## Why You Probably Don't Need MCP

While MCP offers elegant standardisation, that [mcp server proliferation](https://steipete.me/posts/2025/essential-reading-august-2025#less-is-more-the-hidden-costs-of-mcp-server-proliferation) creates a "tragedy of the commons in your context window". Everyone wants their tool included, but the cumulative effect makes the AI assistant worse, not better.

### Context Window Economics

Every MCP server and tool consumes precious tokens from your LLM's limited context window. Our GitHub example above exposed 90 tools to the agent—that's 90 tool schemas taking up space that could be used for actual code, documentation, or reasoning. This creates a trade-off: adding more tools means less space for the work that actually matters.

### The Allocation Paradox

> "Less is more. The more you allocate into the context window of an LLM… the worse the outcomes you're going to get." — Geoffrey Huntley

Each additional tool increases cognitive overhead for the LLM. With too many options, the model must spend reasoning capacity deciding *which* tool to use rather than focusing on solving your actual problem. This paradoxically makes the agent less effective, not more capable.

### Tool Proliferation Problems

Multiple similar tools create non-deterministic behavior. When you have overlapping capabilities (like Github MCP's `read_file` alongside the custom tool's `read_file`), the LLM struggles to choose between them. This confusion leads to:
- Inconsistent tool selection across similar tasks
- Wasted reasoning tokens evaluating redundant options
- Unpredictable agent behavior that's hard to debug

### Security Nightmares

Third-party MCP servers introduce supply chain risks and potential attack vectors. Malicious actors could inject context that manipulates AI behavior, creating a new class of security vulnerabilities:
- Unvetted code running with your agent's permissions
- Potential for prompt injection through tool descriptions
- Enterprise compliance issues with unaudited dependencies

### The Better Approach

**Start with custom tools first.** As this tutorial demonstrates, implementing tools is surprisingly straightforward—just a few lines of Python with the `@tool` decorator. Custom tools give you:
- Full control over functionality and security
- Minimal context window overhead (only what you need)
- Easy debugging and maintenance
- No external dependencies or supply chain risks

**If you must use MCP:**
- Limit servers to essential, irreplaceable functionality
- Prefer first-party integrations from trusted vendors
- Consider dynamically enabling/disabling servers based on specific workflow stages rather than loading everything at once
- Regularly audit which tools are actually being used and remove the rest


## Wrap up

You can find the full example in `main.py`. Our AI agent `HAL` weighs in at just over 300 lines of code, yet it can read files, write content, make precise edits, and execute shell commands. The power comes not from complex orchestration, but from the sophisticated reasoning capabilities of current-generation LLMs combined with a simple, well-structured tool interface.

While this agent can solve real-world problems, it's still in its infancy. There is a lot of grunt work needed to make it very useful, such as [better prompts](https://github.com/jingkaihe/kodelet/blob/main/pkg/tools/bash.go#L39-L167), human and LLM friendly output formatting, [sub-agent orchestration](https://github.com/jingkaihe/kodelet/blob/main/pkg/tools/subagent.go), conversation persistency & forking, image recognition and multi-llm support. The remarkable aspect is that with proper guidance, the agent can evolve and extend itself.